In [43]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [44]:
base_model=MobileNetV2(weights='imagenet', include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

In [45]:
model = Model(inputs=base_model.input, outputs=preds)

In [46]:
batch_size = 8

In [47]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 validation_split=0.3) #included in our dependencies

In [60]:
train_generator = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/dataset123/train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)

X, y = train_generator.next()

# train_generator = ImageDataGenerator(rescale=1. / 255)
# itr = train_datagen.flow_from_directory(
# "/Users/audreatabadero/Downloads/dataset123/train",
#     color_mode='rgb',
# target_size=(224, 224),
# batch_size=batch_size,
#     subset='training',
#     shuffle=True,
# class_mode='categorical')

# X, y = itr.next()

Found 181 images belonging to 3 classes.


In [74]:
# val_generator = ImageDataGenerator(rescale = 1./255)
val_generator = train_datagen.flow_from_directory('/Users/audreatabadero/Downloads/dataset123/validation',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  
                                                  shuffle=True)

# val_generator = train_datagen.flow_from_directory("/Users/audreatabadero/Downloads/dataset123/validation", target_size=(224,224), batch_size=batch_size, subset='validation', shuffle=True, color_mode='rgb', class_mode='categorical')

Found 74 images belonging to 3 classes.


In [75]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [87]:
callbacks = [ModelCheckpoint('/Users/audreatabadero/Downloads/dataset123/checkpoint/weights.{epoch:02d}_{val_loss:.4f}_{val_acc:.4f}.h5')]

In [88]:
# model.fit_generator(generator=train_generator,
#                     steps_per_epoch = train_generator.samples // batch_size,
#                     validation_data=val_generator,
#                     validation_steps = val_generator.samples // batch_size,
#                     callbacks=callbacks,
#                     epochs=20,
#                     )

r = model.fit(X,y, batch_size = batch_size, epochs=2, validation_data=val_generator, callbacks=callbacks)

Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 3.6406e-04 - accuracy: 1.0000 - val_loss: 1.2506 - val_accuracy: 0.5405


KeyError: 'Failed to format this callback filepath: "/Users/audreatabadero/Downloads/dataset123/checkpoint/weights.{epoch:02d}_{val_loss:.4f}_{val_acc:.1}.h5". Reason: \'val_acc\''

In [81]:
best_model = keras.models.load_model('model_chkpt/weights.07_0.1931_0.9254.h5')

OSError: SavedModel file does not exist at: model_chkpt/weights.07_0.1931_0.9254.h5/{saved_model.pbtxt|saved_model.pb}

In [12]:
import tensorflowjs as tfjs

In [13]:
tfjs.converters.save_keras_model(best_model, 'model_output/')

In [39]:
model.predict(np.random.rand(1, 224, 224, 3))

array([[0.31375   , 0.00212419, 0.68412584]], dtype=float32)